In [ ]:
# Creating an expirement

import azureml.core
print("Ready to use Azure ML", azureml.core.VERSION)

import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

import os
# Create a folder for the pipeline step files
experiment_folder = 'covid_exp'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

In [ ]:
# Import Covid Data

run = Run.get_context()

with urllib.request.urlopen("https://api.covid19india.org/data.json") as url:
    s = url.read() 
    json_data = json.loads(s)
    fulldata = pd.DataFrame.from_dict(json_normalize(json_data["cases_time_series"]), orient='columns')
data = fulldata.iloc[:,4:]
convert_dict = {'totalconfirmed': int, 
                'totaldeceased': int,
                'totalrecovered': int,
            
               } 
splitval = int(len(data) * 0.75)
data = data.astype(convert_dict) 
train=data[0:splitval] 
test=data[splitval+1:]

In [ ]:
# Write forecast data to local file

 forecast_df = model.predict() #may vary according to the model you are using
 
#My SIR Model forecasts the Confirmed and Recovered cases count for next one week from today
forecast = {}
forecast['confirmed'] = forecast_df['confirmed']
forecast['recovered'] = forecast_df['recovered']

#Write the forecast data in local file covid_pred.txt
local_file_name =  'covid_pred.txt'
local_path=os.path.abspath(os.path.curdir)
full_path_to_file =os.path.join(local_path, 'covid_pred.txt')    
        
with open(full_path_to_file, 'w') as outfile:
    outfile.write(str(forecast))

In [ ]:
# Upload forecast data to blob

with open(full_path_to_file, 'w') as outfile:
    outfile.write(str(forecast))

print("Temp file = " + full_path_to_file)
# File to upload
file_to_upload = full_path_to_file

sas_url = "" Paste your BLOB URL 
 # Start upload
with open(full_path_to_file , 'rb') as fh:
        response = requests.put(sas_url,
                                data=fh,
                                headers={
                                            'content-type': 'application/octet-stream',
                                            'x-ms-blob-type': 'BlockBlob'
                                        },
                                params={'file': full_path_to_file}
                                )
           

run.complete()

In [ ]:
# Create Pipeline

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "covidcompute"

# Verify that cluster exists
try:
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If not, create it
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4,
                                                           idle_seconds_before_scaledown=1800)
    pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

pipeline_cluster.wait_for_completion(show_output=True)

In [ ]:
# Configure pipeline

from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration

# Create a Python environment for the experiment
covid_env = Environment("covid-env")
covid_env.python.user_managed_dependencies = False # Let Azure ML manage dependencies
covid_env.docker.enabled = True # Use a docker container

# Create a set of package dependencies
covid_packages = CondaDependencies.create(conda_packages=['scikit-learn','pandas','scipy','numpy'],
                                             pip_packages=['azureml-sdk','azure-storage-blob'])

# Add the dependencies to the environment
covid_env.python.conda_dependencies = covid_packages

# Register the environment (just in case you want to use it again)
covid_env.register(workspace=ws)
registered_env = Environment.get(ws, 'covid-env')

# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

# Use the compute you created above. 
pipeline_run_config.target = pipeline_cluster

# Assign the environment to the run configuration
pipeline_run_config.environment = registered_env

print ("Run configuration created.")from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration

# Create a Python environment for the experiment
covid_env = Environment("covid-env")
covid_env.python.user_managed_dependencies = False # Let Azure ML manage dependencies
covid_env.docker.enabled = True # Use a docker container

# Create a set of package dependencies
covid_packages = CondaDependencies.create(conda_packages=['scikit-learn','pandas','scipy','numpy'],
                                             pip_packages=['azureml-sdk','azure-storage-blob'])

# Add the dependencies to the environment
covid_env.python.conda_dependencies = covid_packages

# Register the environment (just in case you want to use it again)
covid_env.register(workspace=ws)
registered_env = Environment.get(ws, 'covid-env')

# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

# Use the compute you created above. 
pipeline_run_config.target = pipeline_cluster

# Assign the environment to the run configuration
pipeline_run_config.environment = registered_env

print ("Run configuration created.")

In [ ]:
# Define pipeline object

from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep
from azureml.train.estimator import Estimator

estimator = Estimator(source_directory=experiment_folder,
                        compute_target = pipeline_cluster,
                        environment_definition=pipeline_run_config.environment,
                        entry_script='prediction.py')

# Step 1, run the estimator to train the model
predict_step = EstimatorStep(name = "Prediction",
                           estimator=estimator, 
                           estimator_entry_script_arguments=[],
                           inputs=[],
                           outputs=[model_folder],
                           compute_target = pipeline_cluster,
                           allow_reuse = True)
print("Pipeline steps defined")

In [ ]:
# Execute pipeline

from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

# Construct the pipeline
pipeline_steps = [predict_step]
pipeline = Pipeline(workspace = ws, steps=pipeline_steps)
print("Pipeline is built.")

# Create an experiment and run the pipeline
experiment = Experiment(workspace = ws, name = 'covid-pipeline')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
print("Pipeline submitted for execution.")

RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion()

published_pipeline = pipeline.publish(name="covid-pipeline",
                                      description="Covid forecast pipeline",
                                      version="1.0")
rest_endpoint = published_pipeline.id
print(rest_endpoint)

In [ ]:
# Schedule pipeline execution (optional)

from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule
recurrence = ScheduleRecurrence(frequency="Day", interval=1)
recurring_schedule = Schedule.create(ws, name="DailyForecastSchedule", 
                            description="Based on time",
                            pipeline_id="",  #replace your pipelineid
                            experiment_name="covid_exp", 
                            recurrence=recurrence)